<a href="https://colab.research.google.com/github/claredavies/AutomaticLearning/blob/main/Assignment1/SupervisedLearningAssignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Libaries

In [140]:
import numpy as np
from google.colab import files
import pandas as pd
from matplotlib import pyplot as plt
from pandas.api.types import is_numeric_dtype
from pandas.api.types import is_string_dtype
import re
import socket
from scipy.stats import zscore
import scipy.stats as stats
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [2]:
pd.set_option('max_rows', 99999)
pd.set_option('max_colwidth', 400)
pd.set_option('display.max_rows', None)

# Read in Files

In [103]:
!git clone https://github.com/claredavies/AutomaticLearning

Cloning into 'AutomaticLearning'...
remote: Enumerating objects: 42, done.
remote: Counting objects: 100% (42/42), done.
remote: Compressing objects: 100% (36/36), done.
remote: Total 42 (delta 6), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (42/42), done.


In [104]:
%ls

AutomaticLearning/  ds_03.csv  im_t_03.txt  im_x_03.txt  im_y_03.txt  README.md


In [105]:
%cd AutomaticLearning/Assignment1/Data/

/content/AutomaticLearning/Assignment1/Data/AutomaticLearning/Assignment1/Data/AutomaticLearning/Assignment1/Data/AutomaticLearning/Assignment1/Data/AutomaticLearning/Assignment1/Data/AutomaticLearning/Assignment1/Data/AutomaticLearning/Assignment1/Data


In [106]:
df = pd.read_csv('ds_03.csv')
df.head(3)

,V01,V02,V03,V04,V05,V06,V07,V08,V09,V10,...,V29,V30,V31,V32,V33,V34,V35,V36,class,target
0,8.82,3.50,10.88,41.70,10.17,NaN,4.18,16.42,3.75,12.87,...,29.30,16.2,10.21,3.84,38.06,NaN,8.47,7.68,1,0.3911
1,12.49,5.08,13.39,35.88,7.02,11.0,6.92,15.35,4.99,6.88,...,68.45,NaN,16.62,4.61,NaN,33.78,9.05,6.12,4,0.4179
2,14.07,3.99,10.87,27.80,9.82,10.6,4.10,NaN,3.34,11.65,...,50.44,NaN,16.84,3.16,39.22,29.64,10.30,11.22,4,0.4090


# Functions

In [9]:
def return_columns_missing_values(df):
  return df.columns[df.isnull().any()]

In [10]:
def missing_value_column_percentage(dataframe, column_name):
  percent_missing = dataframe.loc[:, column_name].isnull().sum() * 100 / len(dataframe)
  return percent_missing

In [71]:
def checkNumberMissingValuesInRowAboveThreshold(dataframe, threshold):
  listMissingValuesAboveThresh = []
  for i in range(len(dataframe.index)):
    resultMissing = dataframe.iloc[i].isnull().sum()
    index = dataframe.iloc[i].index
    if resultMissing > threshold:
      listMissingValuesAboveThresh.append(i)
  # print(listMissingValuesAboveThresh)
  df = dataframe.drop(listMissingValuesAboveThresh, inplace=False)
  return df

In [102]:
def checkZScoreHigherThreshold(dataframe, threshold):
  listOverThreshold = []
  dfZValues = dataframe.apply(stats.zscore)
  for col in dataframe.columns:
    if col != "class":
      found = dfZValues.index[dfZValues[col]>threshold].tolist()
      listOverThreshold = listOverThreshold + found
  listOverThreshold = list(dict.fromkeys(listOverThreshold))
  # print(listOverThreshold)
  df = dataframe.drop(listOverThreshold, inplace=False)
  return df

# T1.a

ai. samples with more than 4 missing feature values have to be dropped,

In [107]:
dfClean = checkNumberMissingValuesInRowAboveThreshold(df, 4)

aii. the remaining missing values have to be filled using the average value, and

In [108]:
dfClean = dfClean.fillna(dfClean.mean())

aiii. samples with at least one feature value with a z-score higher than 3 (i.e. an outlier) have to be discarded.

In [109]:
dfClean = checkZScoreHigherThreshold(dfClean, 3)

# T1.b

It is suspected that not all 36 features are needed for a successful classification. To remove unnecessary
dimensions and reduce the computation time, it is decided to keep only those features that account for
up to 95% of the variance of the data.

In [112]:
Y = dfClean['class']
x = dfClean.drop("class", axis=1)

In [133]:
def findWantedKforPCA(dataframe, threshold):
  for i in range(len(dataframe.columns)):
    pca = PCA(n_components=i)
    principalComponents1 = pca.fit_transform(x)
    variance = sum(pca.explained_variance_ratio_)
    if variance > threshold:
      return i
  print("unable to met threshold")
  return 0

In [138]:
n = findWantedKforPCA(x, 0.95)
print("number of features needed for 95% variance:  " + str(n))
pca = PCA(n_components=n)
principalComponents = pca.fit_transform(x)

pca.explained_variance_ratio_
print(sum(pca.explained_variance_ratio_))

number of features needed for 95% variance:  14
0.9500589971569404


In [141]:
principalComponentsDfX = pd.DataFrame(data = principalComponents
             , columns = ['principal component 1', 'principal component 2','principal component 3', 'principal component 4','principal component 5','principal component 6','principal component 7','principal component 8','principal component 9','principal component 10',
                          'principal component 11','principal component 12','principal component 13', 'principal component 14'])

principalComponentsDfX.tail()

,principal component 1,principal component 2,principal component 3,principal component 4,principal component 5,principal component 6,principal component 7,principal component 8,principal component 9,principal component 10,principal component 11,principal component 12,principal component 13,principal component 14
918,10.517648,8.849201,-10.580086,16.411439,7.663826,-15.744594,8.478717,-9.244069,-0.783213,-0.249700,2.251781,-4.002941,5.771763,1.415364
919,3.628870,0.804786,-12.244010,2.127585,14.357679,-1.606728,8.616712,13.178789,-1.285940,-1.847383,7.393568,-1.256401,0.891386,-2.310283
920,-38.292032,35.411712,-1.974346,-8.600256,-1.253000,-4.488747,-7.452548,7.342709,4.758287,-9.356994,1.554277,3.106598,-2.924636,-0.096985
921,41.345595,-17.492213,-5.649815,2.941931,-2.678740,3.914276,-11.742453,0.371268,-16.114405,-13.910372,-3.600487,-1.383102,-4.089620,0.302808
922,-19.948580,13.821654,-0.024228,-13.221582,0.761577,8.282948,2.931496,-9.340232,16.107987,5.192718,-2.260403,4.466118,0.201214,0.345831


# T1.c

Normalize the data using mu-sigma standardization.

In [142]:
standardizer = StandardScaler()
principalComponentsDfX = standardizer.fit_transform(principalComponentsDfX)

In [145]:
principalComponentsDfX = pd.DataFrame(data = principalComponents
             , columns = ['principal component 1', 'principal component 2','principal component 3', 'principal component 4','principal component 5','principal component 6','principal component 7','principal component 8','principal component 9','principal component 10',
                          'principal component 11','principal component 12','principal component 13', 'principal component 14'])

In [146]:
print(principalComponentsDfX.head())

   principal component 1  principal component 2  principal component 3  \
0               5.985768              13.511855              -6.627281   
1              -6.031639              -4.220891              26.847907   
2              -3.018996              -5.640555               6.549905   
3              10.872085              -4.132533              10.631467   
4             -16.545417               2.249046               7.041152   

   principal component 4  principal component 5  principal component 6  \
0              -2.526279               4.579433              -6.217815   
1               6.683919               2.884829              -2.070621   
2              -4.626191              -5.800534              -1.484647   
3               3.596578              19.045837               5.124192   
4              10.930672             -19.820846             -11.340658   

   principal component 7  principal component 8  principal component 9  \
0              -8.093888            

# T1.d